In [15]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'g:\2025 - Ashoka\Ashoka25_Env\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import timm
import numpy as np
import random
import copy

In [17]:
# Set random seed
sit_number = 123456
torch.manual_seed(sit_number)
np.random.seed(sit_number)
random.seed(sit_number)

In [18]:
# 1. Transformasi dan augmentasi data dengan random_state dan modifikasi augmentasi
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=20, scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [19]:
# 2. Path dataset dan DataLoader
train_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/train"
val_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/val"
test_dir = "NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test"

train_data = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_data = datasets.ImageFolder(root=val_dir, transform=val_test_transform)
test_data = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# Menggunakan resampling untuk menangani ketidakseimbangan kelas
targets = [label for _, label in train_data]
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(targets), y=targets)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Dataloader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Output dataset counts
print(f'Train dataset size: {len(train_data)}')
print(f'Validation dataset size: {len(val_data)}')
print(f'Test dataset size: {len(test_data)}')

Train dataset size: 101
Validation dataset size: 33
Test dataset size: 37


# ConvnextV2

In [20]:
# 3. model ConvNeXt V2
model = timm.create_model('convnextv2_base', pretrained=True)

# Adaptive Pooling untuk memastikan output dari convolutional layer selalu memiliki ukuran tetap
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [21]:
# Set device ke GPU (jika tersedia)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer dengan learning rate yang berbeda untuk layer yang berbeda
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.stages.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)


In [22]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [23]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)

    # Initialize history lists to store metrics
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate and store training loss and accuracy for the epoch
        epoch_train_loss = running_loss / len(train_loader)
        train_loss_history.append(epoch_train_loss)
        # You need to calculate and store training accuracy here as well

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        # Store validation loss and accuracy for the epoch
        val_loss_history.append(val_loss / len(val_loader))
        val_accuracy_history.append(accuracy)


        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/convnextV2_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history

In [24]:
# 7. Evaluasi Model pada Data Uji
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    specificity = tn / (tn + fp)

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')


In [25]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [26]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.3961
Accuracy: 0.7273, Precision: 0.9286, Recall: 0.6190, Specificity: 0.9167, F1 Score: 0.7429
Epoch 2/100
---------------
Val Loss: 0.3410
Accuracy: 0.7273, Precision: 0.8000, Recall: 0.7619, Specificity: 0.6667, F1 Score: 0.7805
Epoch 3/100
---------------
Val Loss: 0.3232
Accuracy: 0.7576, Precision: 0.8421, Recall: 0.7619, Specificity: 0.7500, F1 Score: 0.8000
Epoch 4/100
---------------
Val Loss: 0.3338
Accuracy: 0.7576, Precision: 0.9333, Recall: 0.6667, Specificity: 0.9167, F1 Score: 0.7778
Epoch 5/100
---------------
Val Loss: 0.3753
Accuracy: 0.6970, Precision: 0.9231, Recall: 0.5714, Specificity: 0.9167, F1 Score: 0.7059
Epoch 6/100
---------------
Val Loss: 0.3790
Accuracy: 0.6970, Precision: 0.9231, Recall: 0.5714, Specificity: 0.9167, F1 Score: 0.7059
Epoch 7/100
---------------
Val Loss: 0.3240
Accuracy: 0.7576, Precision: 0.9333, Recall: 0.6667, Specificity: 0.9167, F1 Score: 0.7778
Epoch 8/100
---------------
Val Loss: 0.2951
Acc

In [27]:
# 9. Load and evaluate the best model on test data
# Initialize the model architecture without loading pretrained weights, as we will load the saved weights
best_model = timm.create_model('convnextv2_base', pretrained=False)
best_model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(best_model.num_features, 2)
)

# Load the saved state_dict from best_model.pth
best_model.load_state_dict(torch.load('NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/convnextV2_atas.pth'))
best_model.to(device)
best_model.eval()

# Evaluate the loaded model on test data
evaluate_model(best_model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\3481304985.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('NorAb_V2_Splits-

Accuracy: 0.8108, Precision: 0.8077, Recall: 0.9130, Specificity: 0.6429, F1 Score: 0.8571


In [28]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm
import numpy as np

# Specify the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Model Initialization and Load Weights
best_model = timm.create_model('convnextv2_base', pretrained=False)
best_model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(best_model.num_features, 2)
)

# Load the saved model weights from the .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/convnextV2_atas.pth'
best_model.load_state_dict(torch.load(model_path, map_location=device))
best_model.to(device)
best_model.eval()

# 2. Transformed Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(best_model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\1768580431.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(model_path, map_l

Confusion Matrix:
[[ 9  5]
 [ 2 21]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8182    0.6429    0.7200        14
      Normal     0.8077    0.9130    0.8571        23

    accuracy                         0.8108        37
   macro avg     0.8129    0.7780    0.7886        37
weighted avg     0.8117    0.8108    0.8053        37

ROC-AUC Score: 0.7780

Class: Abnormal
True Positives (TP): 9
False Positives (FP): 5
False Negatives (FN): 2
True Negatives (TN): 21

Class: Normal
True Positives (TP): 21
False Positives (FP): 2
False Negatives (FN): 5
True Negatives (TN): 9

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8182
Recall: 0.6429
F1-Score: 0.7200
Support: 14.0

Normal:
Precision: 0.8077
Recall: 0.9130
F1-Score: 0.8571
Support: 23.0



## Resnet50

In [29]:
from torchvision import models

In [30]:
# 3. Model ResNet50
model = models.resnet50(pretrained=True)

# Replacing fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.0001},
    {'params': model.conv1.parameters(), 'lr': 0.00001},
    {'params': model.layer1.parameters(), 'lr': 0.00001},
    {'params': model.layer2.parameters(), 'lr': 0.00001},
    {'params': model.layer3.parameters(), 'lr': 0.00001},
    {'params': model.layer4.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [32]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [33]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    # Initialize history lists
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    # Early stopping setup
    early_stopping = EarlyStopping(patience=10)

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * inputs.size(0)

            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_train_loss / total_train
        train_accuracy = correct_train / total_train

        # Validation
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / total_val
        val_accuracy = correct_val / total_val

        # Append to history
        train_loss_history.append(train_loss)
        val_loss_history.append(val_loss)
        train_accuracy_history.append(train_accuracy)
        val_accuracy_history.append(val_accuracy)

        # Print results
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Update scheduler
        scheduler.step(val_loss)

        # Early stopping check
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Save the best model weights
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/ResNet50_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)

    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history

In [34]:
# 7. Evaluasi Model pada Data Uji
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    specificity = tn / (tn + fp)

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')
    return accuracy, precision, recall, specificity, f1

In [35]:
# Running training
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [36]:
model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
Train Loss: 0.6686, Train Accuracy: 0.5941
Val Loss: 0.7177, Val Accuracy: 0.5758
Epoch 2/100
Train Loss: 0.7371, Train Accuracy: 0.5545
Val Loss: 0.6929, Val Accuracy: 0.5455
Epoch 3/100
Train Loss: 0.6695, Train Accuracy: 0.5941
Val Loss: 0.6754, Val Accuracy: 0.7273
Epoch 4/100
Train Loss: 0.6653, Train Accuracy: 0.6436
Val Loss: 0.6682, Val Accuracy: 0.6667
Epoch 5/100
Train Loss: 0.6395, Train Accuracy: 0.6733
Val Loss: 0.6663, Val Accuracy: 0.6970
Epoch 6/100
Train Loss: 0.6826, Train Accuracy: 0.5743
Val Loss: 0.6602, Val Accuracy: 0.6667
Epoch 7/100
Train Loss: 0.6820, Train Accuracy: 0.6634
Val Loss: 0.6472, Val Accuracy: 0.6970
Epoch 8/100
Train Loss: 0.6523, Train Accuracy: 0.6436
Val Loss: 0.6265, Val Accuracy: 0.7576
Epoch 9/100
Train Loss: 0.6244, Train Accuracy: 0.7129
Val Loss: 0.6099, Val Accuracy: 0.7273
Epoch 10/100
Train Loss: 0.6115, Train Accuracy: 0.6634
Val Loss: 0.5977, Val Accuracy: 0.7273
Epoch 11/100
Train Loss: 0.6171, Train Accuracy: 0.6832
Val

In [37]:
# Evaluation on test data
evaluate_model(model, test_loader)

Accuracy: 0.8919, Precision: 0.9524, Recall: 0.8696, Specificity: 0.9286, F1 Score: 0.9091


(0.8918918918918919,
 0.9523809523809523,
 0.8695652173913043,
 0.9285714285714286,
 0.9090909090909091)

In [38]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

# Specify the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Load Model ResNet50 without pretrained weights
model = models.resnet50(pretrained=False)

# Replace fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# 2. Load the saved model weight
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/ResNet50_atas.pth'
state_dict = torch.load(model_path, map_location=device, weights_only=True)

new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('fc.') and not (k.endswith('.1.weight') or k.endswith('.1.bias')):
        new_state_dict[k.replace('fc.', 'fc.1.')] = v
    else:
        new_state_dict[k] = v

# Load the customized state_dict into the model
model.load_state_dict(new_state_dict, strict=False)

# Move the model to the device
model.to(device)
model.eval()


# 3. Transformed Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 6. Model Evaluation on Test Data
evaluate_model(model, test_loader)

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Confusion Matrix:
[[13  1]
 [ 3 20]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8125    0.9286    0.8667        14
      Normal     0.9524    0.8696    0.9091        23

    accuracy                         0.8919        37
   macro avg     0.8824    0.8991    0.8879        37
weighted avg     0.8995    0.8919    0.8930        37

ROC-AUC Score: 0.8991

Class: Abnormal
True Positives (TP): 13
False Positives (FP): 1
False Negatives (FN): 3
True Negatives (TN): 20

Class: Normal
True Positives (TP): 20
False Positives (FP): 3
False Negatives (FN): 1
True Negatives (TN): 13

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8125
Recall: 0.9286
F1-Score: 0.8667
Support: 14.0

Normal:
Precision: 0.9524
Recall: 0.8696
F1-Score: 0.9091
Support: 23.0



# ResnetRS

In [39]:
# 3. Model ResNetRS
model = timm.create_model('resnetrs50', pretrained=True)

# Update the head model for two-class classification
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [40]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

criterion = nn.CrossEntropyLoss(weight=class_weights)

In [41]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [42]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/ResnetRS_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [43]:
# 7. Model Evaluation on Test Data
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

In [44]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [45]:
# 8. Train and Evaluate the Model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.5167
Accuracy: 0.6364, Precision: 0.6364, Recall: 1.0000, F1 Score: 0.7778
Epoch 2/100
---------------
Val Loss: 0.4697
Accuracy: 0.6667, Precision: 0.6562, Recall: 1.0000, F1 Score: 0.7925
Epoch 3/100
---------------
Val Loss: 0.4353
Accuracy: 0.7273, Precision: 0.7143, Recall: 0.9524, F1 Score: 0.8163
Epoch 4/100
---------------
Val Loss: 0.4029
Accuracy: 0.7273, Precision: 0.7500, Recall: 0.8571, F1 Score: 0.8000
Epoch 5/100
---------------
Val Loss: 0.3787
Accuracy: 0.7273, Precision: 0.7500, Recall: 0.8571, F1 Score: 0.8000
Epoch 6/100
---------------
Val Loss: 0.3600
Accuracy: 0.6970, Precision: 0.7200, Recall: 0.8571, F1 Score: 0.7826
Epoch 7/100
---------------
Val Loss: 0.3479
Accuracy: 0.6970, Precision: 0.7200, Recall: 0.8571, F1 Score: 0.7826
Epoch 8/100
---------------
Val Loss: 0.3445
Accuracy: 0.6970, Precision: 0.7200, Recall: 0.8571, F1 Score: 0.7826
Epoch 9/100
---------------
Val Loss: 0.3461
Accuracy: 0.6970, Precision: 0.7200, Recall: 0.8571, F1 Score: 

In [46]:
evaluate_model(trained_model, test_loader)

Accuracy: 0.7838, Precision: 0.7586, Recall: 0.9565, F1 Score: 0.8462


In [47]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

# Specify the device to run the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. Model ResNetRS
model = timm.create_model('resnetrs50', pretrained=True)

model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# 2. Load the saved model weight
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/ResnetRS_atas.pth'
state_dict = torch.load(model_path, map_location=device, weights_only=True)

new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('fc.') and not (k.endswith('.1.weight') or k.endswith('.1.bias')):
        new_state_dict[k.replace('fc.', 'fc.1.')] = v
    else:
        new_state_dict[k] = v

# Load the customized state_dict into the model
model.load_state_dict(new_state_dict, strict=False)

# Move the model to the device
model.to(device)
model.eval()


# 3. Transformed Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN for each class, with Abnormal first
    tp_abnormal, fp_normal, fn_abnormal, tn_normal = cm.ravel()

    print("Class: Abnormal")
    print(f"True Positives (TP): {tp_abnormal}")
    print(f"False Positives (FP): {fp_normal}")
    print(f"False Negatives (FN): {fn_abnormal}")
    print(f"True Negatives (TN): {tn_normal}")
    print()

    print("Class: Normal")
    print(f"True Positives (TP): {tn_normal}")
    print(f"False Positives (FP): {fn_abnormal}")
    print(f"False Negatives (FN): {fp_normal}")
    print(f"True Negatives (TN): {tp_abnormal}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 6. Model Evaluation on Test Data
evaluate_model(model, test_loader)

Confusion Matrix:
[[ 7  7]
 [ 1 22]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.8750    0.5000    0.6364        14
      Normal     0.7586    0.9565    0.8462        23

    accuracy                         0.7838        37
   macro avg     0.8168    0.7283    0.7413        37
weighted avg     0.8027    0.7838    0.7668        37

ROC-AUC Score: 0.7283

Class: Abnormal
True Positives (TP): 7
False Positives (FP): 7
False Negatives (FN): 1
True Negatives (TN): 22

Class: Normal
True Positives (TP): 22
False Positives (FP): 1
False Negatives (FN): 7
True Negatives (TN): 7

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.8750
Recall: 0.5000
F1-Score: 0.6364
Support: 14.0

Normal:
Precision: 0.7586
Recall: 0.9565
F1-Score: 0.8462
Support: 23.0



# DeiT

In [103]:
# Import DeiT Model from timm
model = timm.create_model('deit_base_patch16_224', pretrained=True)

# Replacing the head for output to two classes
model.head = nn.Sequential(
    nn.Linear(model.num_features, 2)
)

for param in model.parameters():
    param.requires_grad = True

In [104]:
# Using device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)

In [105]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [110]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        # Saving the best model weights
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [112]:
# Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation phase
        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculating evaluation metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        # Update scheduler dan cek early stopping
        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Load the best weights to the model
    model.load_state_dict(early_stopping.best_model_wts)

    # Save the best model to file
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/DeiT_atas.pth')
    print("Model terbaik telah disimpan ke path yang ditentukan.")

    return model


In [113]:
# Training model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.6383
Accuracy: 0.5455, Precision: 0.8000, Recall: 0.3810, Specificity: 0.8333, F1 Score: 0.5161
Epoch 2/100
---------------
Val Loss: 0.4864
Accuracy: 0.7576, Precision: 0.8095, Recall: 0.8095, Specificity: 0.6667, F1 Score: 0.8095
Epoch 3/100
---------------
Val Loss: 0.3846
Accuracy: 0.7576, Precision: 0.8095, Recall: 0.8095, Specificity: 0.6667, F1 Score: 0.8095
Epoch 4/100
---------------
Val Loss: 0.3389
Accuracy: 0.6970, Precision: 0.7200, Recall: 0.8571, Specificity: 0.4167, F1 Score: 0.7826
Epoch 5/100
---------------
Val Loss: 0.3356
Accuracy: 0.7273, Precision: 0.7308, Recall: 0.9048, Specificity: 0.4167, F1 Score: 0.8085
Epoch 6/100
---------------
Val Loss: 0.3093
Accuracy: 0.7879, Precision: 0.8889, Recall: 0.7619, Specificity: 0.8333, F1 Score: 0.8205
Epoch 7/100
---------------
Val Loss: 0.3047
Accuracy: 0.7273, Precision: 0.7308, Recall: 0.9048, Specificity: 0.4167, F1 Score: 0.8085
Epoch 8/100
---------------
Val Loss: 0.2890
Acc

In [114]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# Specify the device to run the model
best_model = timm.create_model('deit_base_patch16_224', pretrained=True)

# Replacing the head for output to two classes
best_model.head = nn.Sequential(
    nn.Linear(best_model.num_features, 2)
)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/DeiT_atas.pth'
best_model.load_state_dict(torch.load(model_path, map_location=device))
best_model.to(device)
best_model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity: {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(best_model, test_loader)

C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\1248775094.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(model_path, map_l

Confusion Matrix:
[[12  2]
 [ 1 22]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.9231    0.8571    0.8889        14
      Normal     0.9167    0.9565    0.9362        23

    accuracy                         0.9189        37
   macro avg     0.9199    0.9068    0.9125        37
weighted avg     0.9191    0.9189    0.9183        37

ROC-AUC Score: 0.9068

Specificity: 0.8571

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.9231
Recall: 0.8571
F1-Score: 0.8889
Support: 14.0

Normal:
Precision: 0.9167
Recall: 0.9565
F1-Score: 0.9362
Support: 23.0



# MFormer

In [118]:
# 3. Model MFormer
# Using timm to load MFormer
model = timm.create_model('mixer_b16_224', pretrained=True)

#  head for two class classification
model.head = nn.Sequential(
    nn.LayerNorm(model.num_features),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

In [119]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Accessing stem parameters instead of patch_embed
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.blocks.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

# Learning rate scheduler with reduction when loss stagnates
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [120]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [121]:
# 6. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/MFormer_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [122]:
# 8. Train model and save the best weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.6117
Accuracy: 0.4242, Precision: 0.6250, Recall: 0.2381, Specificity: 0.7500, F1 Score: 0.3448
Epoch 2/100
---------------
Val Loss: 0.5762
Accuracy: 0.4848, Precision: 0.7500, Recall: 0.2857, Specificity: 0.8333, F1 Score: 0.4138
Epoch 3/100
---------------
Val Loss: 0.6023
Accuracy: 0.4242, Precision: 0.7500, Recall: 0.1429, Specificity: 0.9167, F1 Score: 0.2400
Epoch 4/100
---------------
Val Loss: 0.5171
Accuracy: 0.5152, Precision: 0.8571, Recall: 0.2857, Specificity: 0.9167, F1 Score: 0.4286
Epoch 5/100
---------------
Val Loss: 0.4765
Accuracy: 0.5455, Precision: 0.8750, Recall: 0.3333, Specificity: 0.9167, F1 Score: 0.4828
Epoch 6/100
---------------
Val Loss: 0.4632
Accuracy: 0.5758, Precision: 0.8889, Recall: 0.3810, Specificity: 0.9167, F1 Score: 0.5333
Epoch 7/100
---------------
Val Loss: 0.4314
Accuracy: 0.6061, Precision: 0.9000, Recall: 0.4286, Specificity: 0.9167, F1 Score: 0.5806
Epoch 8/100
---------------
Val Loss: 0.3958
Acc

In [123]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 1. Model MFormer
model = timm.create_model('mixer_b16_224', pretrained=True)

# head for two class classification
model.head = nn.Sequential(
    nn.LayerNorm(model.num_features),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/MFormer_atas.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)

C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\3577069856.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[11  3]
 [ 5 18]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.6875    0.7857    0.7333        14
      Normal     0.8571    0.7826    0.8182        23

    accuracy                         0.7838        37
   macro avg     0.7723    0.7842    0.7758        37
weighted avg     0.7930    0.7838    0.7861        37

ROC-AUC Score: 0.7842

Specificity : 0.7857

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.6875
Recall: 0.7857
F1-Score: 0.7333
Support: 14.0

Normal:
Precision: 0.8571
Recall: 0.7826
F1-Score: 0.8182
Support: 23.0



# Convnext

In [61]:
# 1. ConvNeXt Model
model = timm.create_model('convnext_base', pretrained=True)

in_features = model.get_classifier().in_features
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

In [62]:
# 2. Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.head.parameters(), 'lr': 0.0001},
    {'params': model.stem.parameters(), 'lr': 0.00001},
    {'params': model.stages.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)


In [63]:
# 3. Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [64]:
# 4. Early Stopping Implementation
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [65]:
# 5. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50):
    early_stopping = EarlyStopping(patience=10)
    train_loss_history, val_loss_history = [], []
    train_accuracy_history, val_accuracy_history = [], []

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 15)

        # Training Phase
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_loss = running_loss / len(train_loader)
        train_accuracy = accuracy_score(all_labels, all_preds)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation Phase
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        # Scheduler Step
        scheduler.step(val_loss)

        # Metrics Logging
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Early Stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Save the best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/Convnext_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [66]:
# 7. Training the Model
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Train Loss: 0.7636, Train Accuracy: 0.5149
Val Loss: 0.5006, Val Accuracy: 0.7273
Epoch 2/100
---------------
Train Loss: 0.6990, Train Accuracy: 0.5050
Val Loss: 0.4987, Val Accuracy: 0.8182
Epoch 3/100
---------------
Train Loss: 0.6123, Train Accuracy: 0.6634
Val Loss: 0.4995, Val Accuracy: 0.7879
Epoch 4/100
---------------
Train Loss: 0.5397, Train Accuracy: 0.6634
Val Loss: 0.4780, Val Accuracy: 0.8485
Epoch 5/100
---------------
Train Loss: 0.5061, Train Accuracy: 0.7228
Val Loss: 0.4558, Val Accuracy: 0.8182
Epoch 6/100
---------------
Train Loss: 0.5451, Train Accuracy: 0.7525
Val Loss: 0.4395, Val Accuracy: 0.7879
Epoch 7/100
---------------
Train Loss: 0.5188, Train Accuracy: 0.7723
Val Loss: 0.4419, Val Accuracy: 0.7879
Epoch 8/100
---------------
Train Loss: 0.4362, Train Accuracy: 0.7723
Val Loss: 0.4353, Val Accuracy: 0.7879
Epoch 9/100
---------------
Train Loss: 0.3956, Train Accuracy: 0.8119
Val Loss: 0.4290, Val Accuracy: 0.8182
Epoch 10/100
---------------
Train Los

In [67]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

model = timm.create_model('convnext_base', pretrained=True)

in_features = model.get_classifier().in_features
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

# Move model to device
model = model.to(device)

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/Convnext_atas.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\2418734703.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[10  4]
 [ 0 23]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     1.0000    0.7143    0.8333        14
      Normal     0.8519    1.0000    0.9200        23

    accuracy                         0.8919        37
   macro avg     0.9259    0.8571    0.8767        37
weighted avg     0.9079    0.8919    0.8872        37

ROC-AUC Score: 0.8571

Specificity : 0.7143

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 1.0000
Recall: 0.7143
F1-Score: 0.8333
Support: 14.0

Normal:
Precision: 0.8519
Recall: 1.0000
F1-Score: 0.9200
Support: 23.0



# RSMamba

In [68]:
# 3. Model RSMamba
model = timm.create_model('resnet50', pretrained=True)

# head for 2 class output
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning
for param in model.parameters():
    param.requires_grad = True

In [69]:
# Optimizer with different learning rates for specific layers
optimizer = optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.0001},
    {'params': [param for name, param in model.named_parameters() if 'fc' not in name], 'lr': 0.00001}
], weight_decay=5e-4)

In [70]:
# 5. Callback Early Stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [71]:
# ipython-input-10-7375c7133d09
import torch

# 6. Training loop dengan Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100, class_weights=None): # Add class_weights as an argument
    # Mendefinisikan device di dalam fungsi train_model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Check if class_weights is provided, and move it to the device if so
    if class_weights is not None:
        class_weights = class_weights.to(device)

    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/RSMamba_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model

In [72]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [73]:
# 8. Train model and save the best weight
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.6975
Accuracy: 0.6364, Precision: 0.6452, Recall: 0.9524, Specificity: 0.0833, F1 Score: 0.7692
Epoch 2/100
---------------
Val Loss: 0.6856
Accuracy: 0.6061, Precision: 0.6250, Recall: 0.9524, Specificity: 0.0000, F1 Score: 0.7547
Epoch 3/100
---------------
Val Loss: 0.6795
Accuracy: 0.5152, Precision: 0.5862, Recall: 0.8095, Specificity: 0.0000, F1 Score: 0.6800
Epoch 4/100
---------------
Val Loss: 0.6811
Accuracy: 0.3939, Precision: 0.5217, Recall: 0.5714, Specificity: 0.0833, F1 Score: 0.5455
Epoch 5/100
---------------
Val Loss: 0.6816
Accuracy: 0.4242, Precision: 0.5500, Recall: 0.5238, Specificity: 0.2500, F1 Score: 0.5366
Epoch 6/100
---------------
Val Loss: 0.6788
Accuracy: 0.3939, Precision: 0.5333, Recall: 0.3810, Specificity: 0.4167, F1 Score: 0.4444
Epoch 7/100
---------------
Val Loss: 0.6686
Accuracy: 0.4545, Precision: 0.6364, Recall: 0.3333, Specificity: 0.6667, F1 Score: 0.4375
Epoch 8/100
---------------
Val Loss: 0.6688
Acc

In [74]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model RSMamba
model = timm.create_model('resnet50', pretrained=True)

# head for 2 class output
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# Enable gradients for fine-tuning
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/RSMamba_atas.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\3659037418.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[ 6  8]
 [ 8 15]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.4286    0.4286    0.4286        14
      Normal     0.6522    0.6522    0.6522        23

    accuracy                         0.5676        37
   macro avg     0.5404    0.5404    0.5404        37
weighted avg     0.5676    0.5676    0.5676        37

ROC-AUC Score: 0.5404

Specificity : 0.4286

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.4286
Recall: 0.4286
F1-Score: 0.4286
Support: 14.0

Normal:
Precision: 0.6522
Recall: 0.6522
F1-Score: 0.6522
Support: 23.0



# Alexnet

In [75]:
# 3. Model AlexNet
from torchvision.models import alexnet

# Initialize AlexNet pretrained model
model = alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [76]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer setup
optimizer = optim.AdamW([
    {'params': model.features.parameters(), 'lr': 0.00001},
    {'params': model.classifier.parameters(), 'lr': 0.0001}
], weight_decay=5e-4)

# Scheduler setup
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

In [77]:
# 5. Training Loop with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        # Training phase
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_accuracy = accuracy_score(all_labels, all_preds)
        train_loss = running_loss / len(train_loader)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(all_labels, all_preds)
        val_loss = val_loss / len(val_loader)
        val_loss_history.append(val_loss)
        val_accuracy_history.append(val_accuracy)

        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        scheduler.step(val_loss)
        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print('Early stopping triggered.')
            break

    # Save the best model weights
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/AlexNet_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)

    return model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history


In [78]:
# 6. Train the AlexNet model
criterion = nn.CrossEntropyLoss(weight=class_weights)
trained_model, train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100
)

Epoch 1/100
---------------
Train Loss: 0.9856, Train Accuracy: 0.4752
Val Loss: 0.6060, Val Accuracy: 0.7576
Epoch 2/100
---------------
Train Loss: 0.6152, Train Accuracy: 0.7822
Val Loss: 0.8460, Val Accuracy: 0.5455
Epoch 3/100
---------------
Train Loss: 0.5033, Train Accuracy: 0.5842
Val Loss: 0.7113, Val Accuracy: 0.5455
Epoch 4/100
---------------
Train Loss: 0.5917, Train Accuracy: 0.7129
Val Loss: 0.3901, Val Accuracy: 0.6970
Epoch 5/100
---------------
Train Loss: 0.4858, Train Accuracy: 0.7426
Val Loss: 0.3125, Val Accuracy: 0.7879
Epoch 6/100
---------------
Train Loss: 0.5185, Train Accuracy: 0.8218
Val Loss: 0.3284, Val Accuracy: 0.7879
Epoch 7/100
---------------
Train Loss: 0.4895, Train Accuracy: 0.7921
Val Loss: 0.3629, Val Accuracy: 0.6970
Epoch 8/100
---------------
Train Loss: 0.4520, Train Accuracy: 0.7426
Val Loss: 0.3585, Val Accuracy: 0.6970
Epoch 9/100
---------------
Train Loss: 0.3345, Train Accuracy: 0.8812
Val Loss: 0.3503, Val Accuracy: 0.6970
Epoch 10/1

In [79]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model AlexNet
from torchvision.models import alexnet

# Initialize AlexNet pretrained model
model = alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/AlexNet_atas.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\2025 - Ashoka\Ashoka25_Env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\2569498155.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blo

Confusion Matrix:
[[11  3]
 [ 6 17]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.6471    0.7857    0.7097        14
      Normal     0.8500    0.7391    0.7907        23

    accuracy                         0.7568        37
   macro avg     0.7485    0.7624    0.7502        37
weighted avg     0.7732    0.7568    0.7600        37

ROC-AUC Score: 0.7624

Specificity : 0.7857

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.6471
Recall: 0.7857
F1-Score: 0.7097
Support: 14.0

Normal:
Precision: 0.8500
Recall: 0.7391
F1-Score: 0.7907
Support: 23.0



# EfficientNet V2

In [80]:
# 3. Model EfficientNetV2
model = timm.create_model('tf_efficientnetv2_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

In [81]:
# 4. Loss function, optimizer, dan scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for classifier and backbone
optimizer = optim.AdamW([
    {'params': model.classifier.parameters(), 'lr': 0.0001},
    {'params': model.conv_stem.parameters(), 'lr': 0.00001},
    {'params': model.blocks.parameters(), 'lr': 0.00001}
], weight_decay=5e-4)

In [82]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function dengan class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [83]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [84]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/EfficientNetV2_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [85]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------
Val Loss: 0.7654
Accuracy: 0.4545, Precision: 0.5652, Recall: 0.6190, Specificity: 0.1667, F1 Score: 0.5909
Epoch 2/100
---------------
Val Loss: 0.7838
Accuracy: 0.4242, Precision: 0.5500, Recall: 0.5238, Specificity: 0.2500, F1 Score: 0.5366
Epoch 3/100
---------------
Val Loss: 0.8085
Accuracy: 0.4545, Precision: 0.6000, Recall: 0.4286, Specificity: 0.5000, F1 Score: 0.5000
Epoch 4/100
---------------
Val Loss: 0.8095
Accuracy: 0.4242, Precision: 0.5714, Recall: 0.3810, Specificity: 0.5000, F1 Score: 0.4571
Epoch 5/100
---------------
Val Loss: 0.7984
Accuracy: 0.3939, Precision: 0.5385, Recall: 0.3333, Specificity: 0.5000, F1 Score: 0.4118
Epoch 6/100
---------------
Val Loss: 0.7880
Accuracy: 0.3636, Precision: 0.5000, Recall: 0.2857, Specificity: 0.5000, F1 Score: 0.3636
Epoch 7/100
---------------
Val Loss: 0.7751
Accuracy: 0.4242, Precision: 0.6000, Recall: 0.2857, Specificity: 0.6667, F1 Score: 0.3871
Epoch 8/100
---------------
Val Loss: 0.7776
Acc

In [86]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model EfficientNetV2
model = timm.create_model('tf_efficientnetv2_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers.
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/EfficientNetV2_atas.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\2873664078.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[11  3]
 [19  4]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.3667    0.7857    0.5000        14
      Normal     0.5714    0.1739    0.2667        23

    accuracy                         0.4054        37
   macro avg     0.4690    0.4798    0.3833        37
weighted avg     0.4940    0.4054    0.3550        37

ROC-AUC Score: 0.4798

Specificity : 0.7857

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.3667
Recall: 0.7857
F1-Score: 0.5000
Support: 14.0

Normal:
Precision: 0.5714
Recall: 0.1739
F1-Score: 0.2667
Support: 23.0



# EfficientNet BO

In [87]:
# 3. Model EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

In [88]:
# 4. Loss function, optimizer, and scheduler setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
class_weights = class_weights.to(device)

# Optimizer with different learning rates for different layers
optimizer = optim.AdamW([
    {'params': model.blocks.parameters(), 'lr': 0.00001},
    {'params': model.classifier.parameters(), 'lr': 0.0001}
], weight_decay=5e-4)

In [89]:
# Scheduler learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

# Loss function dengan class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [90]:
# 5. Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered.')
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())

In [91]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100):
    early_stopping = EarlyStopping(patience=10)
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 15)

        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        val_loss = 0.0
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=1)
        recall = recall_score(all_labels, all_preds, average='binary')
        f1 = f1_score(all_labels, all_preds, average='binary')

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        specificity = tn / (tn + fp)

        print(f'Val Loss: {val_loss / len(val_loader):.4f}')
        print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Specificity: {specificity:.4f}, F1 Score: {f1:.4f}')

        scheduler.step(val_loss / len(val_loader))
        early_stopping(val_loss / len(val_loader), model)
        if early_stopping.early_stop:
            print('Training stopped early')
            break

    # Save best model
    torch.save(early_stopping.best_model_wts, 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/EfficientNetBO_atas.pth')
    model.load_state_dict(early_stopping.best_model_wts)
    return model


In [92]:
# 8. Train model and save the best weights
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)

Epoch 1/100
---------------


Val Loss: 0.8255
Accuracy: 0.4545, Precision: 0.7143, Recall: 0.2381, Specificity: 0.8333, F1 Score: 0.3571
Epoch 2/100
---------------
Val Loss: 0.8518
Accuracy: 0.5758, Precision: 0.8889, Recall: 0.3810, Specificity: 0.9167, F1 Score: 0.5333
Epoch 3/100
---------------
Val Loss: 0.8454
Accuracy: 0.6061, Precision: 0.9000, Recall: 0.4286, Specificity: 0.9167, F1 Score: 0.5806
Epoch 4/100
---------------
Val Loss: 0.8224
Accuracy: 0.6061, Precision: 0.9000, Recall: 0.4286, Specificity: 0.9167, F1 Score: 0.5806
Epoch 5/100
---------------
Val Loss: 0.8069
Accuracy: 0.6061, Precision: 0.9000, Recall: 0.4286, Specificity: 0.9167, F1 Score: 0.5806
Epoch 6/100
---------------
Val Loss: 0.7925
Accuracy: 0.5758, Precision: 0.8182, Recall: 0.4286, Specificity: 0.8333, F1 Score: 0.5625
Epoch 7/100
---------------
Val Loss: 0.7891
Accuracy: 0.6061, Precision: 0.9000, Recall: 0.4286, Specificity: 0.9167, F1 Score: 0.5806
Epoch 8/100
---------------
Val Loss: 0.7795
Accuracy: 0.6061, Precision: 0.

In [93]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import timm

# 3. Model EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True)

# head for 2 class classification
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.num_features, 2)
)

# Enable gradients for fine-tuning all layers
for param in model.parameters():
    param.requires_grad = True

# Load saved model weights from .pth file
model_path = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/modelAtas/EfficientNetBO_atas.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 2. Transform Data for Test Data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Test Data
test_dir = 'NorAb_V2_Splits-20250120T155744Z-001/NorAb_V2_Splits/Sisi Atas/test'
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 4. Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    print()

    # Classification Report with "Abnormal" first
    target_names = ['Abnormal', 'Normal']
    report = classification_report(all_labels, all_preds, target_names=target_names, digits=4)
    print("Classification Report:")
    print(report)

    # ROC-AUC Score
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print()

    # Extract TP, FP, FN, TN
    tn, fp, fn, tp = cm.ravel()

    # Specificity Calculation
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Specificity : {specificity:.4f}")
    print()

    # Detailed Classification Report with custom formatting
    precision_abnormal, recall_abnormal, f1_abnormal, support_abnormal = precision_score(all_labels, all_preds, pos_label=0), recall_score(all_labels, all_preds, pos_label=0), f1_score(all_labels, all_preds, pos_label=0), cm[0].sum()
    precision_normal, recall_normal, f1_normal, support_normal = precision_score(all_labels, all_preds, pos_label=1), recall_score(all_labels, all_preds, pos_label=1), f1_score(all_labels, all_preds, pos_label=1), cm[1].sum()

    print("Detailed Classification Report (with 4 decimal precision):\n")
    print("Abnormal:")
    print(f"Precision: {precision_abnormal:.4f}")
    print(f"Recall: {recall_abnormal:.4f}")
    print(f"F1-Score: {f1_abnormal:.4f}")
    print(f"Support: {support_abnormal:.1f}")
    print()

    print("Normal:")
    print(f"Precision: {precision_normal:.4f}")
    print(f"Recall: {recall_normal:.4f}")
    print(f"F1-Score: {f1_normal:.4f}")
    print(f"Support: {support_normal:.1f}")
    print()

# 5. Model Evaluation on Test Data
evaluate_model(model, test_loader)


C:\Users\Mahasiswa\AppData\Local\Temp\ipykernel_28496\2638039379.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Confusion Matrix:
[[13  1]
 [10 13]]

Classification Report:
              precision    recall  f1-score   support

    Abnormal     0.5652    0.9286    0.7027        14
      Normal     0.9286    0.5652    0.7027        23

    accuracy                         0.7027        37
   macro avg     0.7469    0.7469    0.7027        37
weighted avg     0.7911    0.7027    0.7027        37

ROC-AUC Score: 0.7469

Specificity : 0.9286

Detailed Classification Report (with 4 decimal precision):

Abnormal:
Precision: 0.5652
Recall: 0.9286
F1-Score: 0.7027
Support: 14.0

Normal:
Precision: 0.9286
Recall: 0.5652
F1-Score: 0.7027
Support: 23.0

